<a href="https://www.kaggle.com/code/vtrackstar/best-defensive-nba-seasons?scriptVersionId=202051601" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nba-aba-baa-stats/Advanced.csv
/kaggle/input/nba-aba-baa-stats/Player Season Info.csv
/kaggle/input/nba-aba-baa-stats/Player Career Info.csv
/kaggle/input/nba-aba-baa-stats/Player Shooting.csv
/kaggle/input/nba-aba-baa-stats/Opponent Stats Per Game.csv
/kaggle/input/nba-aba-baa-stats/Player Totals.csv
/kaggle/input/nba-aba-baa-stats/Opponent Stats Per 100 Poss.csv
/kaggle/input/nba-aba-baa-stats/Per 100 Poss.csv
/kaggle/input/nba-aba-baa-stats/Team Summaries.csv
/kaggle/input/nba-aba-baa-stats/Player Per Game.csv
/kaggle/input/nba-aba-baa-stats/All-Star Selections.csv
/kaggle/input/nba-aba-baa-stats/End of Season Teams (Voting).csv
/kaggle/input/nba-aba-baa-stats/Player Play By Play.csv
/kaggle/input/nba-aba-baa-stats/End of Season Teams.csv
/kaggle/input/nba-aba-baa-stats/Team Stats Per 100 Poss.csv
/kaggle/input/nba-aba-baa-stats/Per 36 Minutes.csv
/kaggle/input/nba-aba-baa-stats/Opponent Totals.csv
/kaggle/input/nba-aba-baa-stats/Team Abbrev.csv
/kaggle/input/nba-aba-b

In [2]:
playerpg = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Per Game.csv')
playerpbp = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Play By Play.csv')
playeraw = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Award Shares.csv')
playersh = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Shooting.csv')
playertot = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Totals.csv')
playeradv = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Advanced.csv')

print(playerpg.columns)
print(playerpbp.columns)
print(playeraw.columns)
print(playersh.columns)
print(playertot.columns)
print(playeradv.columns)

Index(['seas_id', 'season', 'player_id', 'player', 'birth_year', 'pos', 'age',
       'experience', 'lg', 'tm', 'g', 'gs', 'mp_per_game', 'fg_per_game',
       'fga_per_game', 'fg_percent', 'x3p_per_game', 'x3pa_per_game',
       'x3p_percent', 'x2p_per_game', 'x2pa_per_game', 'x2p_percent',
       'e_fg_percent', 'ft_per_game', 'fta_per_game', 'ft_percent',
       'orb_per_game', 'drb_per_game', 'trb_per_game', 'ast_per_game',
       'stl_per_game', 'blk_per_game', 'tov_per_game', 'pf_per_game',
       'pts_per_game'],
      dtype='object')
Index(['seas_id', 'season', 'player_id', 'player', 'birth_year', 'pos', 'age',
       'experience', 'lg', 'tm', 'g', 'mp', 'pg_percent', 'sg_percent',
       'sf_percent', 'pf_percent', 'c_percent',
       'on_court_plus_minus_per_100_poss', 'net_plus_minus_per_100_poss',
       'bad_pass_turnover', 'lost_ball_turnover', 'shooting_foul_committed',
       'offensive_foul_committed', 'shooting_foul_drawn',
       'offensive_foul_drawn', 'points_gener

In [3]:
defpg = playertot[['seas_id', 'season', 'player', 'pos', 'tm', 'g', 'drb', 'stl', 'blk', 'pf']]
defpbp = playerpbp[['seas_id', 'on_court_plus_minus_per_100_poss', 'net_plus_minus_per_100_poss', 'shooting_foul_committed', 'offensive_foul_drawn']]
defadv = playeradv[['seas_id', 'dws', 'dbpm']]

defpr = defpg.merge(defpbp, on='seas_id', how='outer')
defplr = defpr.merge(defadv, on='seas_id', how='outer')
defplar = defplr.dropna()
defplayr = defplar[defplar['g'] >= 60]
dfplayer = defplayr.copy()
dfplayer.head()

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,on_court_plus_minus_per_100_poss,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,dws,dbpm
16713,16714,2000,A.C. Green,PF,LAL,82,326.0,53.0,18.0,127,7.7,-0.8,53.0,18.0,3.3,0.8
16715,16716,2000,Aaron McKie,SG,PHI,82,199.0,108.0,18.0,194,0.2,-2.0,75.0,8.0,3.0,2.2
16716,16717,2000,Aaron Williams,PF,WAS,81,250.0,41.0,92.0,234,1.6,8.1,120.0,16.0,1.6,0.4
16717,16718,2000,Adam Keefe,C,UTA,62,91.0,17.0,13.0,90,-2.0,-7.9,29.0,5.0,0.8,0.4
16718,16719,2000,Adonal Foyle,C,GSW,76,250.0,26.0,136.0,218,-9.7,-0.9,116.0,18.0,1.2,-0.1


In [4]:
award = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Award Shares.csv')
defpoy = award[award['award'] == 'dpoy']
defdpoy = defpoy.copy()
print(defdpoy.columns)

Index(['season', 'award', 'player', 'age', 'tm', 'first', 'pts_won', 'pts_max',
       'share', 'winner', 'seas_id', 'player_id'],
      dtype='object')


In [5]:
dpoy = defpoy[['seas_id', 'award', 'share', 'winner']]
defplayer = dfplayer.merge(dpoy, on='seas_id', how='inner')
defplayer['winner'] = defplayer['winner'].replace({'True': 1, 'False': 0}).astype(int)
defplayer.head()

/tmp/ipykernel_17/3451578862.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  defplayer['winner'] = defplayer['winner'].replace({'True': 1, 'False': 0}).astype(int)


,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,on_court_plus_minus_per_100_poss,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,dws,dbpm,award,share,winner
0,16726,2000,Alonzo Mourning,C,MIA,79,538.0,40.0,294.0,308,7.1,5.8,95.0,24.0,5.6,2.2,dpoy,0.512,1
1,16739,2000,Anthony Mason,SF,CHH,82,554.0,74.0,29.0,220,3.4,1.8,109.0,25.0,3.9,0.7,dpoy,0.008,0
2,16769,2000,Bo Outlaw,PF,ORL,82,323.0,113.0,148.0,203,6.0,13.1,72.0,30.0,4.0,3.6,dpoy,0.008,0
3,16823,2000,Clifford Robinson,SF,PHO,80,254.0,90.0,61.0,239,4.6,-4.2,91.0,38.0,3.9,0.4,dpoy,0.008,0
4,16867,2000,Dikembe Mutombo,C,ATL,82,853.0,27.0,269.0,248,-1.5,13.1,103.0,19.0,4.3,1.5,dpoy,0.091,0


In [6]:
unique_values = defplayer['pos'].unique()
print(unique_values)

['C' 'SF' 'PF' 'SG' 'PG' 'PF-C' 'SF-PF']


In [7]:
def categorize(pos):
    if pos in ['SG', 'PG', 'PG-SG', 'SG-PG']:
        return 'Guard'
    elif pos in ['SF', 'SF-PF', 'PF-SF', 'SF-SG', 'SG-SF', 'SF-C', 'SG-PG-SF', 'SG-PF']:
        return 'Wing'
    else:
        return 'Big'

defplayer['pos_group'] = defplayer['pos'].apply(categorize)

defplayer.head()

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,on_court_plus_minus_per_100_poss,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,dws,dbpm,award,share,winner,pos_group
0,16726,2000,Alonzo Mourning,C,MIA,79,538.0,40.0,294.0,308,7.1,5.8,95.0,24.0,5.6,2.2,dpoy,0.512,1,Big
1,16739,2000,Anthony Mason,SF,CHH,82,554.0,74.0,29.0,220,3.4,1.8,109.0,25.0,3.9,0.7,dpoy,0.008,0,Wing
2,16769,2000,Bo Outlaw,PF,ORL,82,323.0,113.0,148.0,203,6.0,13.1,72.0,30.0,4.0,3.6,dpoy,0.008,0,Big
3,16823,2000,Clifford Robinson,SF,PHO,80,254.0,90.0,61.0,239,4.6,-4.2,91.0,38.0,3.9,0.4,dpoy,0.008,0,Wing
4,16867,2000,Dikembe Mutombo,C,ATL,82,853.0,27.0,269.0,248,-1.5,13.1,103.0,19.0,4.3,1.5,dpoy,0.091,0,Big


In [8]:
df1 = defplayer[defplayer['pos_group'] == 'Guard']
defguard = df1.copy()
defguard['score'] = (
    defguard['share'] * 12 +
    defguard['winner'] * 10 +
    defguard['drb'] * 0.002 +
    defguard['dbpm'] * 4 +
    defguard['dws'] * 6 +
    defguard['stl'] * 0.4 +
    defguard['blk'] * 0.3 +
    defguard['shooting_foul_committed'] * -0.3 +
    defguard['offensive_foul_drawn'] * 0.2
)
defguard.head()

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,...,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,dws,dbpm,award,share,winner,pos_group,score
5,16886,2000,Eddie Jones,SG,CHH,72,262.0,192.0,49.0,176,...,11.7,57.0,54.0,4.4,2.0,dpoy,0.091,0,Guard,121.216
6,16910,2000,Gary Payton,PG,SEA,82,429.0,153.0,18.0,178,...,14.7,41.0,30.0,3.5,0.7,dpoy,0.033,0,Guard,85.354
7,16948,2000,Jason Kidd,PG,PHO,67,387.0,134.0,28.0,148,...,-0.9,53.0,17.0,4.8,2.0,dpoy,0.008,0,Guard,87.170
10,17000,2000,Kobe Bryant,SG,LAL,66,308.0,106.0,62.0,220,...,4.9,94.0,13.0,4.5,1.6,dpoy,0.033,0,Guard,69.812
19,20074,2006,Eddie Jones,SG,MEM,75,244.0,131.0,27.0,201,...,7.2,73.0,24.0,4.2,2.3,dpoy,0.002,0,Guard,78.312


In [9]:
df2 = defplayer[defplayer['pos_group'] == 'Wing']
defwing = df2.copy()
defwing['score'] = (
    defwing['share'] * 30 +
    defwing['winner'] * 7 +
    defwing['dws'] * 3 +
    defwing['stl'] * 0.5 +
    defwing['blk'] * 0.2 +
    defwing['shooting_foul_committed'] * -0.75 +
    defwing['offensive_foul_drawn'] * 0.2
)
defwing.head()

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,...,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,dws,dbpm,award,share,winner,pos_group,score
1,16739,2000,Anthony Mason,SF,CHH,82,554.0,74.0,29.0,220,...,1.8,109.0,25.0,3.9,0.7,dpoy,0.008,0,Wing,-22.01
3,16823,2000,Clifford Robinson,SF,PHO,80,254.0,90.0,61.0,239,...,-4.2,91.0,38.0,3.9,0.4,dpoy,0.008,0,Wing,8.49
8,16989,2000,Kendall Gill,SF,NJN,76,201.0,139.0,41.0,211,...,-1.0,78.0,14.0,2.1,0.7,dpoy,0.008,0,Wing,28.54
11,17136,2000,Scottie Pippen,SF,POR,82,399.0,117.0,41.0,208,...,0.5,82.0,26.0,4.4,1.7,dpoy,0.008,0,Wing,23.84
14,19923,2006,Andre Iguodala,SF,PHI,82,364.0,135.0,21.0,198,...,10.5,110.0,26.0,2.6,0.4,dpoy,0.003,0,Wing,2.29


In [10]:
df3 = defplayer[defplayer['pos_group'] == 'Big']
defbig = df3.copy()
defbig['score'] = (
    defbig['share'] * 12 +
    defbig['winner'] * 5 +
    defbig['drb'] * 0.4 +
    defbig['dws'] * 1.5 +
    defbig['stl'] * 0.2 +
    defbig['blk'] * 0.5 +
    defbig['shooting_foul_committed'] * -0.25 +
    defbig['offensive_foul_drawn'] * 0.1
)
defbig.head()

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,...,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,dws,dbpm,award,share,winner,pos_group,score
0,16726,2000,Alonzo Mourning,C,MIA,79,538.0,40.0,294.0,308,...,5.8,95.0,24.0,5.6,2.2,dpoy,0.512,1,Big,368.394
2,16769,2000,Bo Outlaw,PF,ORL,82,323.0,113.0,148.0,203,...,13.1,72.0,30.0,4.0,3.6,dpoy,0.008,0,Big,216.896
4,16867,2000,Dikembe Mutombo,C,ATL,82,853.0,27.0,269.0,248,...,13.1,103.0,19.0,4.3,1.5,dpoy,0.091,0,Big,464.792
9,16994,2000,Kevin Garnett,PF,MIN,81,733.0,120.0,126.0,205,...,11.6,70.0,15.0,5.6,1.7,dpoy,0.017,0,Big,372.804
12,17142,2000,Shaquille O'Neal,C,LAL,79,742.0,36.0,239.0,255,...,10.9,144.0,12.0,7.0,2.0,dpoy,0.174,0,Big,401.288


In [11]:
sortguard = defguard.sort_values(by='score', ascending=False)
sortguard.head()

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,...,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,dws,dbpm,award,share,winner,pos_group,score
80,21691,2009,Chris Paul,PG,NOH,78,363.0,216.0,10.0,212,...,19.6,53.0,22.0,5.0,3.8,dpoy,0.082,0,Guard,124.810
61,21089,2008,Chris Paul,PG,NOH,80,259.0,217.0,4.0,185,...,9.1,36.0,16.0,4.6,3.2,dpoy,0.039,0,Guard,121.786
5,16886,2000,Eddie Jones,SG,CHH,72,262.0,192.0,49.0,176,...,11.7,57.0,54.0,4.4,2.0,dpoy,0.091,0,Guard,121.216
82,21760,2009,Dwyane Wade,SG,MIA,79,309.0,173.0,106.0,178,...,14.2,80.0,14.0,4.4,2.4,dpoy,0.151,0,Guard,118.230
121,22841,2011,Chris Paul,PG,NOH,80,289.0,188.0,5.0,196,...,13.9,52.0,35.0,4.4,2.8,dpoy,0.022,0,Guard,106.542


In [12]:
sortwing = defwing.sort_values(by='score', ascending=False)
sortwing.head()

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,...,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,dws,dbpm,award,share,winner,pos_group,score
215,26060,2016,Kawhi Leonard,SF,SAS,72,398.0,128.0,71.0,133,...,7.1,50.0,11.0,5.5,3.0,dpoy,0.842,1,Wing,91.66
197,25447,2015,Kawhi Leonard,SF,SAS,64,376.0,148.0,48.0,128,...,11.2,53.0,5.0,4.4,3.1,dpoy,0.516,1,Wing,80.53
254,28111,2019,Paul George,SF,OKC,77,523.0,170.0,34.0,214,...,16.5,61.0,30.0,4.9,1.6,dpoy,0.300,0,Wing,75.75
88,21909,2009,LeBron James,SF,CLE,81,507.0,137.0,93.0,139,...,21.2,55.0,13.0,6.5,3.7,dpoy,0.249,0,Wing,75.42
225,26641,2017,Kawhi Leonard,SF,SAS,74,350.0,133.0,55.0,122,...,1.6,48.0,13.0,4.7,2.2,dpoy,0.364,0,Wing,69.12


In [13]:
sortbig = defbig.sort_values(by='score', ascending=False)
sortbig.head()

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,...,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,dws,dbpm,award,share,winner,pos_group,score
4,16867,2000,Dikembe Mutombo,C,ATL,82,853.0,27.0,269.0,248,...,13.1,103.0,19.0,4.3,1.5,dpoy,0.091,0,Big,464.792
67,21350,2008,Marcus Camby,C,DEN,79,807.0,85.0,285.0,216,...,1.0,145.0,33.0,5.9,3.3,dpoy,0.287,0,Big,461.644
103,22328,2010,Dwight Howard,C,ORL,82,798.0,75.0,228.0,287,...,9.4,134.0,3.0,7.1,2.8,dpoy,0.944,1,Big,441.978
62,21157,2008,Dwight Howard,C,ORL,82,882.0,74.0,176.0,274,...,8.0,133.0,8.0,6.4,1.2,dpoy,0.039,0,Big,433.218
81,21759,2009,Dwight Howard,C,ORL,79,757.0,77.0,231.0,270,...,8.2,120.0,8.0,7.6,2.4,dpoy,0.911,1,Big,431.832


In [14]:
best_guard = sortguard.drop_duplicates(subset='player', keep='first')
best_guard.head(10)

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,...,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,dws,dbpm,award,share,winner,pos_group,score
80,21691,2009,Chris Paul,PG,NOH,78,363.0,216.0,10.0,212,...,19.6,53.0,22.0,5.0,3.8,dpoy,0.082,0,Guard,124.810
5,16886,2000,Eddie Jones,SG,CHH,72,262.0,192.0,49.0,176,...,11.7,57.0,54.0,4.4,2.0,dpoy,0.091,0,Guard,121.216
82,21760,2009,Dwyane Wade,SG,MIA,79,309.0,173.0,106.0,178,...,14.2,80.0,14.0,4.4,2.4,dpoy,0.151,0,Guard,118.230
245,27554,2018,Victor Oladipo,SG,IND,75,343.0,177.0,57.0,175,...,13.7,71.0,10.0,4.0,1.7,dpoy,0.002,0,Guard,100.110
166,24325,2013,Mike Conley,PG,MEM,80,182.0,174.0,24.0,171,...,16.1,69.0,26.0,4.7,2.3,dpoy,0.002,0,Guard,99.088
114,22601,2010,Rajon Rondo,PG,BOS,81,260.0,189.0,11.0,198,...,2.4,62.0,12.0,4.7,1.4,dpoy,0.090,0,Guard,98.100
275,30183,2022,Marcus Smart,PG,BOS,71,228.0,119.0,18.0,160,...,4.7,62.0,46.0,3.7,1.7,dpoy,0.514,1,Guard,89.224
303,31769,2024,Shai Gilgeous-Alexander,PG,OKC,75,350.0,150.0,67.0,184,...,11.4,99.0,11.0,4.2,2.3,dpoy,0.012,0,Guard,87.844
7,16948,2000,Jason Kidd,PG,PHO,67,387.0,134.0,28.0,148,...,-0.9,53.0,17.0,4.8,2.0,dpoy,0.008,0,Guard,87.170
6,16910,2000,Gary Payton,PG,SEA,82,429.0,153.0,18.0,178,...,14.7,41.0,30.0,3.5,0.7,dpoy,0.033,0,Guard,85.354


In [15]:
best_wing = sortwing.drop_duplicates(subset='player', keep='first')
best_wing.head(10)

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,...,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,dws,dbpm,award,share,winner,pos_group,score
215,26060,2016,Kawhi Leonard,SF,SAS,72,398.0,128.0,71.0,133,...,7.1,50.0,11.0,5.5,3.0,dpoy,0.842,1,Wing,91.66
254,28111,2019,Paul George,SF,OKC,77,523.0,170.0,34.0,214,...,16.5,61.0,30.0,4.9,1.6,dpoy,0.300,0,Wing,75.75
88,21909,2009,LeBron James,SF,CLE,81,507.0,137.0,93.0,139,...,21.2,55.0,13.0,6.5,3.7,dpoy,0.249,0,Wing,75.42
15,19926,2006,Andrei Kirilenko,SF,UTA,69,391.0,102.0,220.0,162,...,12.3,67.0,24.0,4.3,3.1,dpoy,0.195,0,Wing,68.30
221,26294,2016,Trevor Ariza,SF,HOU,81,299.0,160.0,26.0,177,...,2.4,56.0,5.0,2.3,0.3,dpoy,0.002,0,Wing,51.16
287,30759,2023,Jimmy Butler,SF,MIA,64,234.0,117.0,21.0,80,...,3.5,32.0,12.0,2.9,2.0,dpoy,0.002,0,Wing,49.86
141,23373,2012,Andre Iguodala,SF,PHI,62,325.0,107.0,30.0,92,...,3.1,43.0,15.0,4.2,2.5,dpoy,0.055,0,Wing,44.50
49,20808,2007,Metta World Peace,SF,SAC,70,352.0,149.0,43.0,205,...,7.6,70.0,16.0,3.1,0.9,dpoy,0.031,0,Wing,44.03
105,22361,2010,Gerald Wallace,SF,CHA,76,612.0,117.0,83.0,194,...,1.8,86.0,44.0,6.2,1.7,dpoy,0.185,0,Wing,43.55
53,20914,2007,Shawn Marion,SF,PHO,80,613.0,156.0,122.0,216,...,8.3,115.0,11.0,5.3,1.9,dpoy,0.143,0,Wing,38.54


In [16]:
best_big = sortbig.drop_duplicates(subset='player', keep='first')
best_big.head(10)

,seas_id,season,player,pos,tm,g,drb,stl,blk,pf,...,net_plus_minus_per_100_poss,shooting_foul_committed,offensive_foul_drawn,dws,dbpm,award,share,winner,pos_group,score
4,16867,2000,Dikembe Mutombo,C,ATL,82,853.0,27.0,269.0,248,...,13.1,103.0,19.0,4.3,1.5,dpoy,0.091,0,Big,464.792
67,21350,2008,Marcus Camby,C,DEN,79,807.0,85.0,285.0,216,...,1.0,145.0,33.0,5.9,3.3,dpoy,0.287,0,Big,461.644
103,22328,2010,Dwight Howard,C,ORL,82,798.0,75.0,228.0,287,...,9.4,134.0,3.0,7.1,2.8,dpoy,0.944,1,Big,441.978
192,25236,2015,DeAndre Jordan,C,LAC,82,829.0,81.0,183.0,245,...,15.4,128.0,4.0,5.4,0.7,dpoy,0.405,0,Big,420.660
232,26937,2018,Andre Drummond,C,DET,78,848.0,114.0,127.0,250,...,-3.1,88.0,8.0,5.7,1.7,dpoy,0.002,0,Big,412.874
12,17142,2000,Shaquille O'Neal,C,LAL,79,742.0,36.0,239.0,255,...,10.9,144.0,12.0,7.0,2.0,dpoy,0.174,0,Big,401.288
270,29529,2021,Rudy Gobert,C,UTA,71,720.0,40.0,190.0,163,...,19.1,81.0,14.0,5.2,2.8,dpoy,0.928,1,Big,396.086
224,26534,2017,Hassan Whiteside,C,MIA,77,795.0,57.0,161.0,226,...,1.9,108.0,12.0,5.3,0.5,dpoy,0.006,0,Big,392.122
44,20742,2007,Kevin Garnett,PF,MIN,76,792.0,89.0,126.0,184,...,14.8,80.0,15.0,4.9,1.7,dpoy,0.011,0,Big,386.582
293,31170,2024,Anthony Davis,C,LAL,76,722.0,91.0,178.0,177,...,6.0,117.0,4.0,4.7,1.7,dpoy,0.168,0,Big,376.216
